### Balance geneal

In [2]:
import yfinance as yf
import pandas as pd
import time

# Leer tickers desde archivo
tickers_df = pd.read_csv("Ticketsvalidos.csv")
tickers = tickers_df.iloc[:, 0].dropna().unique().tolist()


# variables monetarias (se escalan a millones); EPS se deja tal cual
monetary_keys = [k for k in income_statement_keys if k not in ("basicEPS", "dilutedEPS")]

registros = []  # acumularemos dicts {Ticker, Year, ...}

for ti in tickers:
    for intento in range(3):
        try:
            ticker = yf.Ticker(ti)
            income_stmt = ticker.income_stmt  # anual

            if income_stmt is None or income_stmt.empty:
                print(f"{ti} no tiene estado de resultados disponible.")
                break

            # Asegurar que las filas estén en el orden de keys; si falta alguna, yfinance no la incluye
            available = [k for k in income_statement_keys if k in income_stmt.index]
            if not available:
                print(f"{ti}: no hay claves reconocidas del income statement.")
                break

            # Iterar por columna (cada columna es una fecha de cierre, típica: 2024-12-31)
            for col in income_stmt.columns:
                # convertir a año
                try:
                    year = pd.to_datetime(col).year
                except Exception:
                    # Si por alguna razón no parsea, lo dejamos como string
                    year = str(col)

                fila = {"Ticker": ti, "Year": year}
                for key in income_statement_keys:
                    if key in income_stmt.index:
                        val = income_stmt.loc[key, col]
                        if pd.notna(val):
                            if key in monetary_keys:
                                fila[key] = round(float(val) / 1_000_000, 2)  # millones
                            else:
                                fila[key] = float(val)  # EPS sin escalar
                        else:
                            fila[key] = None
                    else:
                        fila[key] = None

                registros.append(fila)

            print(f"{ti} procesado.")
            break

        except Exception as e:
            msg = str(e).lower()
            if any(err in msg for err in ["too many requests", "rate limit"]):
                print(f"Rate limit en {ti}, esperando 10s (intento {intento+1}/3)")
                time.sleep(10)
            else:
                print(f"Error en {ti}: {e}")
                break
    time.sleep(1)

# DataFrame final en formato ancho (Ticker, Year, variables…)
df_income = pd.DataFrame(registros)
df_income = df_income.sort_values(["Ticker", "Year"]).reset_index(drop=True)

# Guardar
df_income.to_csv("EstadoResultados_Anual.csv", index=False)
print("Estado de resultados anual guardado en EstadoResultados_Anual.csv")


Se truncaron las últimas líneas 5000 del resultado de transmisión.
CSGG.DE no tiene estado de resultados disponible.
ECHO.DE no tiene estado de resultados disponible.
COPMA.DE no tiene estado de resultados disponible.
3ER.DE no tiene estado de resultados disponible.
CE2G.DE no tiene estado de resultados disponible.
ECPG.DE no tiene estado de resultados disponible.
BDN.DE no tiene estado de resultados disponible.
DAMG.DE no tiene estado de resultados disponible.
LOUDG.DE no tiene estado de resultados disponible.
DFTKN.DE no tiene estado de resultados disponible.
HNLG.DE no tiene estado de resultados disponible.
DRWG.DE no tiene estado de resultados disponible.
BE4A.DE no tiene estado de resultados disponible.
BGSN.DE no tiene estado de resultados disponible.
EUZG.DE no tiene estado de resultados disponible.
E4CG.DE no tiene estado de resultados disponible.
EING_P.DE no tiene estado de resultados disponible.
EE.DE no tiene estado de resultados disponible.
ELGG.DE no tiene estado de resul

KeyError: 'Ticker'